# COMPARE DIFFERENT IMPLEMENTATIONS

### IMPORTS

Perform imports and check environment.

In [1]:
import pandas as pd
import torch
from multiprocessing import Pool, cpu_count
import numpy as np
from src.input.write_files import write_int_matrix_file
from src.input.read_files import find_files, numbers_from_file_name, normal_sec_size_int_from_file_name, number_from_text
from datetime import datetime
from src.algorithm.circuit_synthesis_vanilla import synthesise_circuit as synthesise_circuit_vanilla
from src.algorithm.circuit_synthesis_torch import synthesise_circuit as synthesise_circuit_torch
from src.algorithm.execution import load_matrix_and_benchmark, load_tensor_and_benchmark
#from src.algorithm.execution import load_matrix_and_benchmark
import csv

print(f'There are {cpu_count()} threads available on this machineee.')
print(f'CUDA is{" " if torch.cuda.is_available() else " not "}available on this machine.')

There are 12 threads available on this machineee.
CUDA is available on this machine.


### INPUT GENERATION PARAMETERS

Defining the parameters for the input generation.

In [2]:
sample_size = 2
lower_size = 100
upper_size = 115
step_size = 5

sizes = np.arange(lower_size, upper_size + step_size, step_size)
samples = np.arange(1, sample_size + 1, 1)

input_gen_processors = 10

input_dir = r'resources\implementation_comparison\input_matrices'
input_file_prefix = 'SingMat'

### INPUT GENERATION

Generate the matrices and save them to files.

In [6]:
input_gen_argument_tuples = ([(size, size * size, rf'..\{input_dir}\{input_file_prefix}_{size}_{sample}.txt') for size in sizes for sample in samples])

[print(argument_tuple) for argument_tuple in input_gen_argument_tuples[:min(10, len(input_gen_argument_tuples))]]

(100, 10000, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_100_1.txt')
(100, 10000, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_100_2.txt')
(105, 11025, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_105_1.txt')
(105, 11025, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_105_2.txt')
(110, 12100, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_110_1.txt')
(110, 12100, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_110_2.txt')
(115, 13225, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_115_1.txt')
(115, 13225, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_115_2.txt')


[None, None, None, None, None, None, None, None]

In [7]:
print(f'Generating {len(input_gen_argument_tuples)} matrices in size range from {lower_size} to {upper_size} ...')

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(write_int_matrix_file, input_gen_argument_tuples)

print(f'Saved matrices to {input_dir} ...')

Generating 8 matrices in size range from 100 to 115 ...
Saved matrices to resources\implementation_comparison\input_matrices ...


### GET FILES TO BENCHMARK

Find relevant files and sort them.

In [3]:
file_regex = rf'({input_file_prefix}).*\.txt'
size_regex = r'_\d+_'
sample_regex = r'_\d+\.'

input_file_names = find_files(rf'..\{input_dir}', file_regex)
input_file_names.sort(key = lambda input_file_name: numbers_from_file_name(input_file_name, size_regex, sample_regex))

[print(input_file_name) for input_file_name in input_file_names[:min(10, len(input_file_names))]]

SingMat_100_1.txt
SingMat_100_2.txt
SingMat_105_1.txt
SingMat_105_2.txt
SingMat_110_1.txt
SingMat_110_2.txt
SingMat_115_1.txt
SingMat_115_2.txt


[None, None, None, None, None, None, None, None]

### GET DIFFERENT SECTION SIZES

Get the normal section sizes and produce input tuples together with the files.

In [4]:
section_sizes_df = pd.read_csv(rf'..\resources\optimization\SecSizes_2_5000.csv')
section_sizes_df = section_sizes_df.loc[(section_sizes_df['size'] >= lower_size) & (section_sizes_df['size'] <= upper_size)][['size', 'norm_sec_size']]
section_sizes_df.head()

,size,norm_sec_size
98,100,3
99,101,3
100,102,3
101,103,3
102,104,3


In [5]:
section_sizes_pivot_df = section_sizes_df.set_index('size').stack().reset_index(name='sec_size').rename(columns={'level_1':'sec_size_type'})
section_sizes_pivot_df.head()

,size,sec_size_type,sec_size
0,100,norm_sec_size,3
1,101,norm_sec_size,3
2,102,norm_sec_size,3
3,103,norm_sec_size,3
4,104,norm_sec_size,3


In [6]:
file_name_arguments = [(number_from_text(input_file_name, size_regex),
                        number_from_text(input_file_name, sample_regex),
                        input_file_name)
                       for input_file_name in input_file_names]
file_name_arguments_df = pd.DataFrame(file_name_arguments, columns =['size', 'sample', 'path'])
file_name_arguments_df.head()

,size,sample,path
0,100,1,SingMat_100_1.txt
1,100,2,SingMat_100_2.txt
2,105,1,SingMat_105_1.txt
3,105,2,SingMat_105_2.txt
4,110,1,SingMat_110_1.txt


In [7]:
benchmark_tuples_df = pd.merge(section_sizes_pivot_df, file_name_arguments_df, on='size', how='inner')[['size', 'sec_size', 'sample', 'path']]
benchmark_tuples_df.head()

,size,sec_size,sample,path
0,100,3,1,SingMat_100_1.txt
1,100,3,2,SingMat_100_2.txt
2,105,3,1,SingMat_105_1.txt
3,105,3,2,SingMat_105_2.txt
4,110,3,1,SingMat_110_1.txt


In [8]:
benchmark_tuples_list = [tuple(benchmark_tuple) for benchmark_tuple in benchmark_tuples_df.values.tolist()]

benchmark_argument_tuples = [(synthesise_circuit_torch,
                              size,
                              sec_size,
                              sample,
                              rf'..\{input_dir}\{input_file_name}')
                             for (size, sec_size, sample, input_file_name) in benchmark_tuples_list]

[print(argument_tuple) for argument_tuple in benchmark_argument_tuples[:min(10, len(benchmark_argument_tuples))]]

(<function synthesise_circuit at 0x000002657D87A280>, 100, 3, 1, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_100_1.txt')
(<function synthesise_circuit at 0x000002657D87A280>, 100, 3, 2, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_100_2.txt')
(<function synthesise_circuit at 0x000002657D87A280>, 105, 3, 1, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_105_1.txt')
(<function synthesise_circuit at 0x000002657D87A280>, 105, 3, 2, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_105_2.txt')
(<function synthesise_circuit at 0x000002657D87A280>, 110, 3, 1, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_110_1.txt')
(<function synthesise_circuit at 0x000002657D87A280>, 110, 3, 2, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_110_2.txt')
(<function synthesise_circuit at 0x000002657D87A280>, 115, 3, 1, '..\\resources\\implementation_comparison\\input_matrices\\SingMat_115_

[None, None, None, None, None, None, None, None]

### DEFINE BENCHMARK PARAMETERS

In [9]:
output_dir = r'resources\implementation_comparison'
output_file_prefix = 'ImplementationComparison'

output_file_fieldnames = ['size', 'run', 'sec_size', 'num_gates', 'process_time', 'initial_rss', 'final_rss']

### BENCHMARK VANILLA

Read each file into a matrix, perform a benchmark and save the results in a file.

In [12]:
vanilla_output_file_postfix = 'Vanilla'
output_file_timestamp = datetime.now().strftime("%d%m%y_%H%M")

#print(f'Benchmarking runtime performance in size range from {lower_size} to {upper_size} in {benchmark_processors} threads ...')

vanilla_result_tuples = []

for benchmark_argument_tuple in benchmark_argument_tuples:

    vanilla_result_tuples.append(load_matrix_and_benchmark(*benchmark_argument_tuple))

print(f"Saving results to '{output_dir} ...")

with open(rf'..\{output_dir}\{output_file_prefix}_{vanilla_output_file_postfix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in vanilla_result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')

Loading matrix from ..\resources\implementation_comparison\input_matrices\SingMat_100_1.txt ...
Benchmarking sample 1 ... matrix size: 100, section size: 3


TypeError: logical_xor(): argument 'input' (position 1) must be Tensor, not list

### BENCHMARK TORCH

Read each file into a tensor, perform a benchmark and save the results in a file.

In [10]:
torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'The PyTorch implementation will be executed on the device {torch_device}.')

The PyTorch implementation will be executed on the device cuda.


In [11]:
torch_output_file_postfix = 'Torch'
output_file_timestamp = datetime.now().strftime("%d%m%y_%H%M")



#print(f'Benchmarking runtime performance in size range from {lower_size} to {upper_size} in {benchmark_processors} threads ...')

torch_result_tuples = []

for benchmark_argument_tuple in benchmark_argument_tuples:

    torch_result_tuples.append(load_tensor_and_benchmark(*benchmark_argument_tuple, torch_device))

print(f"Saving results to '{output_dir} ...")

with open(rf'..\{output_dir}\{output_file_prefix}_{torch_output_file_postfix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in torch_result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')


Loading tensor from ..\resources\implementation_comparison\input_matrices\SingMat_100_1.txt ...
Tensor allocated on cuda ...
Benchmarking sample 1 ... matrix size: 100, section size: 3
Loading tensor from ..\resources\implementation_comparison\input_matrices\SingMat_100_2.txt ...
Tensor allocated on cuda ...
Benchmarking sample 2 ... matrix size: 100, section size: 3
Loading tensor from ..\resources\implementation_comparison\input_matrices\SingMat_105_1.txt ...
Tensor allocated on cuda ...
Benchmarking sample 1 ... matrix size: 105, section size: 3
Loading tensor from ..\resources\implementation_comparison\input_matrices\SingMat_105_2.txt ...
Tensor allocated on cuda ...
Benchmarking sample 2 ... matrix size: 105, section size: 3
Loading tensor from ..\resources\implementation_comparison\input_matrices\SingMat_110_1.txt ...
Tensor allocated on cuda ...
Benchmarking sample 1 ... matrix size: 110, section size: 3
Loading tensor from ..\resources\implementation_comparison\input_matrices\S